Since using imshow function from cv2 was not working properly, I imported pyplot from matplotlib in order to check the image. Rest of the methods are exclusively from libraries numpy and cv2. 

In [1]:
import numpy as np
import cv2

%matplotlib inline
from matplotlib import pyplot as plt

The following preprocessing is identical to method 1. <br>
Read and grayscale both images upsampled.png and HR.png. <br>
Resize the __image_gt__ and make __image_l__.<br> 
Implement functions mse and PSNR. <br>
Compute PSNR for __image_h__. <br>

In [2]:
image_h= cv2.imread('upsampled.png')
image_h=cv2.cvtColor(image_h,cv2.COLOR_BGR2GRAY).astype(np.float32)

image_gt=cv2.imread('HR.png')
image_gt=cv2.cvtColor(image_gt,cv2.COLOR_BGR2GRAY).astype(np.float32)

height, width=image_h.shape
image_l=cv2.resize(image_gt,(height//4,width//4))

def mse(img):
    MSE=np.sum(np.square(image_gt-img))/(height*width)
    return MSE
def PSNR(img):
    psnr=10*np.log10(255*255/mse(img))
    return psnr
print(PSNR(image_h))

18.00910086146031


Compute the gradient and Laplacian of __image_l__. Then, show the Laplacian of the image. 

In [7]:
gamma=6
grad_image_l=np.linalg.norm(np.abs(cv2.Sobel(image_h,-1,1,0))+np.abs(cv2.Sobel(image_h,-1,0,1)))+1e-10
laplacian_image_l=cv2.Laplacian(image_h,-1)

Subtract the gradient of __image_l__ and Laplacian, and obtain sharp edge image by clipping. 

In [14]:
sharp_edge=grad_image_l-np.linalg.norm(laplacian_image_l)
sharp_edge=np.clip(sharp_edge,a_min=0, a_max=1.0)
laplacian_image_T=gamma*laplacian_image_l*(sharp_edge/grad_image_l)

Gradient Descent with prior

In [9]:
beta=0.0001
MAX_ITERATION=1000
lr=0.1

for i in range(MAX_ITERATION):
    image_T_=cv2.Laplacian(image_h,-1)
    image_d=cv2.resize(image_h,(height//4, width//4))
    grad=cv2.resize(image_d-image_l,(height,width))-beta*(image_T_-laplacian_image_T)
    image_h=image_h-np.dot(lr,grad)
#     loss=np.square(image_l-image_d)+beta*(np.square(grad-))
    
cv2.imwrite('super-resolution-method2.png',image_h)
print(PSNR(image_h))

20.711061950730247
